In [1]:
from baseline_cnn import *

In [2]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [3]:
# Check if your system supports CUDA
use_cuda = torch.cuda.is_available()

# Setup GPU optimization if CUDA is supported
if use_cuda:
    computing_device = torch.device("cuda")
    extras = {"num_workers": 1, "pin_memory": True}
    print("CUDA is supported")
else: # Otherwise, train on the CPU
    computing_device = torch.device("cpu")
    extras = False
    print("CUDA NOT supported")

net=Nnet().to(computing_device)
net.apply(weights_init)

# Print the model
print(net)

#loss criteria are defined in the torch.nn package
criterion = nn.CrossEntropyLoss()


#Instantiate the gradient descent optimizer - use Adam optimizer with default parameters
optimizer = optim.Adam(net.parameters(),lr = 0.001)

CUDA NOT supported
Nnet(
  (main): Sequential(
    (0): Conv2d(3, 21, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): ReLU(inplace=True)
    (2): Conv2d(21, 20, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): ReLU(inplace=True)
    (5): Conv2d(20, 15, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(15, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): ReLU(inplace=True)
    (8): Conv2d(15, 7, kernel_size=(5, 5), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(7, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
  )
  (fc): Sequential(
    (0): Linear(in_features=1575, out_features=300, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=300, out_features=201, bias=True)
  )
)


In [6]:
transform = transforms.Compose([transforms.Resize(256), transforms.CenterCrop(256), transforms.ToTensor()])
dataset = loader('train.csv', "", transform=transform)
batch_size = 64
validation_split = .2
shuffle_dataset = True
random_seed= 42

# Creating data indices for training and validation splits:
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
                                           sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                                sampler=valid_sampler)

In [7]:
def loadStatistics(labels, outputs):
    decisions = outputs.argmax(axis=1)

    for i in range (1,201):
        truePositives[i-1] += int(torch.sum((decisions == i) & (decisions == labels)))
        falsePositives[i-1] += int(torch.sum((decisions == i) & (decisions != labels)))
        falseNegatives[i-1] += int(torch.sum((labels == i) & (decisions != labels)))


def printStatistics():
    acc = accuracy(truePositives, batch_size)
    prec = precision(truePositives, falsePositives)
    rec = recall(truePositives, falseNegatives)
    bal = bcr(prec, rec)

    print("True Positives: ", truePositives)    
    print("False Positives: ", falsePositives)
    print("False Negatives: ", falseNegatives)
    print("Accuracy: ", acc)
    print("Precision: ", prec)
    print("Recall: ", rec)
    print("Balanced Classification Rate: ", bal)


In [8]:
# Track the loss across training
total_loss = []
avg_minibatch_loss = []
N = 50

validation_loss = []

truePositives = np.zeros(200)
falsePositives = np.zeros(200)
falseNegatives = np.zeros(200)

for epoch in range(50):
    N_minibatch_loss = 0.0

    # Get the next minibatch of images, labels for training
    for minibatch_count, (images, labels) in enumerate(train_loader, 0):
        print("mini_batch", minibatch_count)
        # Zero out the stored gradient (buffer) from the previous iteration
        optimizer.zero_grad()
        # Put the minibatch data in CUDA Tensors and run on the GPU if supported
        images, labels = images.to(computing_device), labels.to(computing_device)
        # Perform the forward pass through the network and compute the loss
        outputs = net(images)
        
        loss = criterion(outputs, labels)
        # Automagically compute the gradients and backpropagate the loss through the network
        loss.backward()

        # Update the weights
        optimizer.step()
        # Add this iteration's loss to the total_loss
        total_loss.append(loss.item())
        N_minibatch_loss += loss
                    
                
        if minibatch_count % N == N-1:
            #Print the loss averaged over the last N mini-batches
            N_minibatch_loss /= N
            print('Epoch %d' % (epoch + 1))
            print('average minibatch %d loss: %.3f' % (minibatch_count+1, N_minibatch_loss))
            print('')
            printStatistics(labels, outputs)

            # Add the averaged loss over N minibatches and reset the counter
            avg_minibatch_loss.append(N_minibatch_loss)
            N_minibatch_loss = 0.0

    print("Finished", epoch + 1, "epochs of training")
    # TODO: Implement validation #with torch.no_grad():
    
    with torch.no_grad():
        valid_loss = 0
        for minibatch_count, (images, labels) in enumerate(validation_loader, 0):
            print("Validation mini_batch", minibatch_count)
            images, labels = images.to(computing_device), labels.to(computing_device)
            outputs = net(images)

            valid_loss += criterion(outputs, labels)
        validation_loss.append(valid_loss)
        
    if epoch >= 5:
        early_stop = True
        for i in range(5):
            if validation_loss[epoch - i] < validation_loss[epoch - i - 1]:
                early_stop = False
        
        if early_stop == True:
            break

mini_batch 0
mini_batch 1
mini_batch 2
mini_batch 3
mini_batch 4
mini_batch 5
mini_batch 6
mini_batch 7
mini_batch 8
mini_batch 9
mini_batch 10
mini_batch 11
mini_batch 12
mini_batch 13
mini_batch 14
mini_batch 15
mini_batch 16
mini_batch 17
mini_batch 18
mini_batch 19
mini_batch 20
mini_batch 21
mini_batch 22
mini_batch 23
mini_batch 24
mini_batch 25
mini_batch 26
mini_batch 27
mini_batch 28
mini_batch 29
mini_batch 30
mini_batch 31
mini_batch 32
mini_batch 33
mini_batch 34
mini_batch 35
mini_batch 36
mini_batch 37
mini_batch 38
mini_batch 39
mini_batch 40
mini_batch 41
mini_batch 42
mini_batch 43
mini_batch 44
mini_batch 45
mini_batch 46
mini_batch 47
mini_batch 48
mini_batch 49
Epoch 1
average minibatch 50 loss: 5.232



TypeError: printStatistics() takes 0 positional arguments but 2 were given

In [9]:
# Load up the test dataset
test_dataset = loader('test.csv',transform=transform)
test_size = len(test_dataset)
test_indices = list(range(test_size))
test_sampler = SubsetRandomSampler(test_indices)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, 
                                           sampler=test_sampler)

In [10]:
# Initialize evaluation metrics
total_test_loss = []
N = 50

truePositives = np.zeros(200)
falsePositives = np.zeros(200)
falseNegatives = np.zeros(200)

In [11]:
with torch.no_grad():

    # Get the next minibatch of images, labels for training
    for minibatch_count, (images, labels) in enumerate(test_loader, 0):
        print("mini_batch", minibatch_count)
        # Put the minibatch data in CUDA Tensors and run on the GPU if supported
        images, labels = images.to(computing_device), labels.to(computing_device)
        # Perform the forward pass through the network and compute the loss
        outputs = net(images)

        loss = criterion(outputs, labels)

        # Add this iteration's loss to the total_loss
        total_test_loss.append(loss.item())

        loadStatistics(labels, outputs)
        
    printStatistics()

mini_batch 0
mini_batch 1
mini_batch 2
mini_batch 3
mini_batch 4
mini_batch 5
mini_batch 6
mini_batch 7
mini_batch 8
mini_batch 9
mini_batch 10
mini_batch 11
mini_batch 12
mini_batch 13
mini_batch 14
mini_batch 15
mini_batch 16
mini_batch 17
mini_batch 18
mini_batch 19
mini_batch 20
mini_batch 21
mini_batch 22
mini_batch 23
mini_batch 24
mini_batch 25
mini_batch 26
mini_batch 27
mini_batch 28
mini_batch 29
mini_batch 30
mini_batch 31
mini_batch 32
mini_batch 33
mini_batch 34
mini_batch 35
mini_batch 36
mini_batch 37
mini_batch 38
mini_batch 39
mini_batch 40
mini_batch 41
mini_batch 42
mini_batch 43
mini_batch 44
mini_batch 45
mini_batch 46
mini_batch 47
mini_batch 48
mini_batch 49
mini_batch 50
True Positives:  [19.  1.  9.  4. 10.  7. 11.  4. 16.  8.  4.  4. 13.  3. 12. 11.  8.  7.
  6.  8.  8.  6.  5. 17.  9.  1.  7. 12. 21.  6.  5.  6. 14.  5. 20. 13.
  8. 11.  6.  8.  7.  2.  4.  7.  4.  8.  7. 11.  5. 10.  5.  6. 13.  9.
  3.  4.  6. 20. 12.  5.  1. 14.  7.  9. 15.  9. 11. 14. 18.